Load libraries.  In this case we need the ndex library, which you can get with "pip install ndex"

In [ ]:
#ndex = __import__("ndex")

In [1]:
import ndex2
import requests
import json
import base64
from ndex2.cx.aspects import ATTRIBUTE_DATA_TYPE

# Connect to NDEx in order to retrieve a network.

In [2]:
anon_ndex = ndex2.client.Ndex2("http://public.ndexbio.org")

Get the "NCI Pathway Interaction Database - Final Revision - Extended Binary SIF" from NDEx, using its UUID. 

In [3]:
#nci_pid_uuid='c0e70804-d848-11e6-86b1-0ac135e8bacf'
nci_pid_uuid='e7e460bc-f71f-11e7-adc1-0ac135e8bacf'

nci_pid = ndex2.create_nice_cx_from_server(server='http://public.ndexbio.org', uuid=nci_pid_uuid)

In [4]:
for k, v in nci_pid.nodeAttributes.items():
    for na in v:
        val_type = na.get_data_type()
        if na.get_data_type() == ATTRIBUTE_DATA_TYPE.FLOAT:
            value = na.get_values()
            na.set_values(str(value))
        elif na.get_data_type() == ATTRIBUTE_DATA_TYPE.INTEGER:
            value = na.get_values()
            na.set_values(str(value))
        elif na.get_data_type() == ATTRIBUTE_DATA_TYPE.DOUBLE:
            value = na.get_values()
            na.set_values(str(value))
        elif na.get_data_type() == ATTRIBUTE_DATA_TYPE.LIST_OF_FLOAT:
            values = na.get_values()
            na.set_values('[' + ','.join(str(e) for e in values) + ']')
        elif na.get_data_type() == ATTRIBUTE_DATA_TYPE.LIST_OF_INTEGER:
            values = na.get_values()
            na.set_values('[' + ','.join(str(e) for e in values) + ']')
        elif na.get_data_type() == ATTRIBUTE_DATA_TYPE.LIST_OF_DOUBLE:
            values = na.get_values()
            na.set_values('[' + ','.join(str(e) for e in values) + ']')
        elif na.get_data_type() == ATTRIBUTE_DATA_TYPE.LIST_OF_STRING:
            values = na.get_values()
            na.set_values('[' + ','.join(values) + ']')
        elif na.get_data_type() == None:
            value = na.get_values()
            na.set_values(str(value))
        else:
            value = na.get_values()
            na.set_values(str(value))
        print(na.get_values())

nci_pid.get_summary()

207
true
0.006904205458073183
[uniprot knowledgebase:B0LPH7,uniprot knowledgebase:Q02156,uniprot knowledgebase:Q32MQ3,uniprot knowledgebase:Q53SL4,uniprot knowledgebase:Q53SM5,uniprot knowledgebase:Q9UE81]
Protein
0
34
true
0.010405545718119417
[uniprot knowledgebase:A1L467,uniprot knowledgebase:B5A932,uniprot knowledgebase:E7EQ94,uniprot knowledgebase:O60366,uniprot knowledgebase:P08581,uniprot knowledgebase:Q12875,uniprot knowledgebase:Q9UDX7,uniprot knowledgebase:Q9UPL8]
Protein
0
648
true
0.0031672995262295757
[uniprot knowledgebase:D0PQI2,uniprot knowledgebase:Q502W5,uniprot knowledgebase:Q5T5M7,uniprot knowledgebase:Q5VTQ9,uniprot knowledgebase:Q5VTR0,uniprot knowledgebase:Q8N8S7,uniprot knowledgebase:Q9NVF3,uniprot knowledgebase:Q9UFB8]
Protein
0
294
true
0.00632344974775492
[chebi:CHEBI:37550]
SmallMolecule
0
29
true
0.011017472922842666
[uniprot knowledgebase:B2R5Z2,uniprot knowledgebase:B4DQI8,uniprot knowledgebase:B4DW74,uniprot knowledgebase:B4DW94,uniprot knowledgebase:P09

u'Name: Neighborhood query result on network - NCI Pathway Interaction Database - Diffusion Demo Copy\nNodes: 157\nEdges: 191\nNode Attributes: 942\nEdge Attributes: 955\n'

Set the first node in the network to have the attribute "diffusion_input" to 1.0. This is the heat that will be diffused through the network to compute ranks and heats for each node.

In [10]:
for k, v in nci_pid.nodeAttributes.items():
    for na in v:
        print(na.to_cx())

{<CX_CONSTANTS.DATA_TYPE: 'd'>: 'integer', <CX_CONSTANTS.EDGE_SOURCE_NODE_ID_OR_SUBNETWORK: 's'>: 796, <CX_CONSTANTS.VALUE: 'v'>: '207', <CX_CONSTANTS.PROPERTY_OF: 'po'>: 1541, <CX_CONSTANTS.NAME: 'n'>: u'diffusion_output_rank'}
{<CX_CONSTANTS.DATA_TYPE: 'd'>: 'boolean', <CX_CONSTANTS.EDGE_SOURCE_NODE_ID_OR_SUBNETWORK: 's'>: 796, <CX_CONSTANTS.VALUE: 'v'>: 'true', <CX_CONSTANTS.PROPERTY_OF: 'po'>: 1541, <CX_CONSTANTS.NAME: 'n'>: u'selected'}
{<CX_CONSTANTS.DATA_TYPE: 'd'>: 'double', <CX_CONSTANTS.EDGE_SOURCE_NODE_ID_OR_SUBNETWORK: 's'>: 796, <CX_CONSTANTS.VALUE: 'v'>: '0.006904205458073183', <CX_CONSTANTS.PROPERTY_OF: 'po'>: 1541, <CX_CONSTANTS.NAME: 'n'>: u'diffusion_output_heat'}
{<CX_CONSTANTS.DATA_TYPE: 'd'>: 'list_of_string', <CX_CONSTANTS.EDGE_SOURCE_NODE_ID_OR_SUBNETWORK: 's'>: 796, <CX_CONSTANTS.VALUE: 'v'>: u'[uniprot knowledgebase:B0LPH7,uniprot knowledgebase:Q02156,uniprot knowledgebase:Q32MQ3,uniprot knowledgebase:Q53SL4,uniprot knowledgebase:Q53SM5,uniprot knowledgebase:Q9

In [5]:
node_id, node = next(iter(nci_pid.get_nodes()))
print(node_id)
print("heat source is " + node.get_name())
nci_pid.set_node_attribute(node, "diffusion_input", 1.0)

1541
heat source is PRKCE


Call the Diffusion service

In [6]:
url = 'http://v3.heat-diffusion.cytoscape.io'
payload = nci_pid.to_cx()
#print(payload)
headers = {'content-type': 'application/json'}
response = requests.post(url, data=json.dumps(payload), headers=headers)
print(response.text)

{
    "data": "",
    "errors": [
        {
            "type": "urn:cytoscape:ci:cxmate:heat-diffusion:500",
            "message": "error parsing required aspect element in nodeAttributes error: json: cannot unmarshal number into Go value of type string",
            "link": "http://github.com/cxmate/cxmate",
            "status": 500
        }
    ]
}



#The 'data' dictionary in the json response is a dictionary with node IDs as keys and, 'rank' and 'heat' as the values. 

See the top 10 most relevant nodes to your query. 

In [ ]:
nos = []
#R = ndex.networkn.NdexGraph(cx=response.json()['data'])
R = ndex2.create_nice_cx_from_cx(response.json()['data'])

for k, v in R.get_nodes():
    node_attr = R.get_node_attribute(v, 'diffusion_output_rank')
    print(node_attr)
    if node_attr is not None and node_attr.get_values() < 10:
        print(node_attr.get_values())
        



#for n in R.nodes_iter():
#    if R.node[n]['diffusion_output_rank'] < 10:
#        R.node[n]['nid'] = n
#        nos.append(R.node[n])
#nos = sorted(nos, key=lambda k: k['diffusion_output_rank'])
#for no in nos:
#    print "id: " + str(no['nid']) + " name: " + no['name'] + " rank: " + str(no['diffusion_output_rank']) + " heat: " + str(no['diffusion_output_heat'])
